<a href="https://colab.research.google.com/github/zhaizeyu/test/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install huggingface_hub
! pip install transformers sentencepiece
!pip install --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.7 MB/s eta 0:00:00


In [11]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-crede

In [2]:
from transformers import AutoTokenizer
import transformers
import torch
import accelerate
model="codellama/CodeLlama-13b-Instruct-hf" ## 感兴趣的可以换成其他模型试试
tokenizer = AutoTokenizer.from_pretrained(model) ## 自动下载并加载模型的分词器



In [ ]:
!pip install accelerate

In [3]:
## 生成本文
pipeline = transformers.pipeline(
    "text-generation", ## 管道类型
    model=model,
    torch_dtype=torch.float16, ## 设置模型的数据类型为 float16。这通常用于加速计算，
## 特别是在支持半精度计算的 GPU 上。如果你只有 CPU，这可能不会有太大影响
    device_map="auto", ## 自动选择CPU或GPU
    #low_cpu_mem_usage=False
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
system ="Provide answers in Python."
user = "Please give me the Python code to return all the Fibonacci numbers under 100."

prompt = f"<s><<SYS>>\n{system}\n<</SYS>>\n\n{user}"

In [14]:
sequences = pipeline(prompt,eos_token_id=tokenizer.eos_token_id,top_k=10, # 概率最高的10个词)

SyntaxError: incomplete input (<ipython-input-14-cae351e95ae9>, line 2)

In [ ]:
sequences = pipeline(
    prompt,
    #do_sample=True, ## 抽样生成文本，确保文本多样性
    top_k=10, # 概率最高的10个词
    #temperature=0.1, #比较稳定，一定的灵活性
    top_p=0.95, #保留累积概率达到 95% 的最高概率词汇
    num_return_sequences=1, #每次只生成一个序列
    eos_token_id=tokenizer.eos_token_id, #指定了“结束符”的标记ID，来自于先前定义的分词器（tokenizer）
## id = 2
    max_length=100,
    #add_special_tokens=False #生成文本时不添加特殊标记（如开始、结束标记）
)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [20]:
for seq in sequences: #sequences变量应该包含了由文本生成管道产生的一个或多个文本序列。
#每个序列都是一个字典（或类似的数据结构），包含了生成的文本和可能的其他信息。
  print(f"Result: {seq['generated_text']}")

Result: <s><<SYS>>
Provide answers in Python.
<</SYS>>

Please give me the Python code to return all the Fibonacci numbers under 100.

